In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics


In [3]:
data = pd.read_csv("healthy.txt", header=0, \
                    delimiter="\t")
data.head(10)

IOError: File healthy.txt does not exist

In [ ]:
data.Intent_1.value_counts()

In [ ]:
#Change User Intent labels to numerical values
data['Intent_1'] = data.Intent_1.map({'Generic': 0 , 'Direct Answer': 1, 'Guide': 2, 'Navigational': 3, 'Resource': 4, 'Transactional': 5 })

#Drop unused columns, null rows, and reindex

health_data = data.drop(['\xef\xbb\xbfCS_ID', 'Keyword_ID', 'Authority'], axis=1).dropna()
health_data = health_data.drop_duplicates('Keyword')
health_data_reindex = health_data.reset_index(drop=True)
health_data_reindex.info()
health_data_reindex.head(10)

In [ ]:
da_other_intents = health_data_reindex[health_data_reindex['Intent_1']>1].copy()
da_other_intents.Intent_1.value_counts()

kw_target = da_other_intents.Intent_1
kw_features = da_other_intents.Keyword

X_train, X_test, Y_train, Y_test = train_test_split(kw_features, kw_target, test_size=0.2, random_state=1) 

#da_other_intents.Intent_1.value_counts()

#train, test = train_test_split(da_other_intents, test_size=0.2, random_state=1)

In [ ]:
from sklearn.linear_model import SGDClassifier

# Classifier pipeline-- CountVectorizer and TfidfTransformer vectorize the keywords and transform them into numerical values
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2', n_iter=5, random_state=42, class_weight='balanced')), ])


#Fit and train model
text_clf_fit = text_clf.fit(X_train, Y_train)

#Output predictions
predictions = text_clf_fit.predict(X_test)

In [4]:
#Classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

NameError: name 'y_test' is not defined

In [5]:
#Undersample Classes
classes = [2,3,4,5]
subsets = []
for i in classes:
    subset = train[train.Intent_1 == i].sample(921)
    subsets.append(subset)
    
train_subset = pd.concat(subsets)

X_train, y_train = train_subset['Keyword'], train_subset['Intent_1']
X_test, y_test = test['Keyword'], test['Intent_1']

NameError: name 'train' is not defined

In [6]:
#LinearSVC
from sklearn.svm import LinearSVC

text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', LinearSVC()), ])

text_clf_fit = text_clf.fit(X_train, y_train)
predictions = text_clf_fit.predict(X_test)

NameError: name 'X_train' is not defined

In [7]:
#Evaluation Block
print metrics.accuracy_score(y_test, predictions)
print metrics.confusion_matrix(y_test, predictions)

cross_val_accuracy = cross_val_score(text_clf, kw_features, kw_target, cv=5).mean()
cross_val_accuracy

cm = metrics.confusion_matrix(y_test,predictions)
cm_df = pd.DataFrame(cm, index=['Predicted GUIDE (2)', 'Predicted NAV (3)', 'Predicted RES (4)', 'Predicted BUY (5)'], 
                     columns=['Actual GUIDE (2)', 'Actual NAV (3)', 'Actual RES (4)', 'Actual BUY (5)'])

cm_df

NameError: name 'y_test' is not defined

In [8]:
#Evaluation Block
print metrics.accuracy_score(y_test, predictions)
print metrics.confusion_matrix(y_test, predictions)

cross_val_accuracy = cross_val_score(text_clf, kw_features, kw_target, cv=5).mean()
cross_val_accuracy

cm = metrics.confusion_matrix(y_test,predictions)
cm_df = pd.DataFrame(cm, index=['Predicted GUIDE (2)', 'Predicted NAV (3)', 'Predicted RES (4)', 'Predicted BUY (5)'], 
                     columns=['Actual GUIDE (2)', 'Actual NAV (3)', 'Actual RES (4)', 'Actual BUY (5)'])

cm_df

NameError: name 'y_test' is not defined

In [9]:
#Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

NameError: name 'y_test' is not defined

#Evaluation Block?
classes = [2,3,4,5]

with open('errors_output.txt', 'w') as w:
    for i in classes:
        errors = X_test[(y_test == i) & (predictions != i)]
        w.write(errors + '\t'+ i )
for i in classes:
    errors = X_test[(y_test == i) & (predictions != i)]
    

errors = X_test[(y_test == 3) & (predictions != 3)]


with open('errors_output.txt', 'w') as w:
    for i in incorrect_DAs:
        w.write(i+'\t'+'False Direct Answer'+'\n')
    for g in incorrect_GENs:
        w.write(g+'\t'+'False Generic'+'\n')

In [10]:
errors = X_test[(y_test == 4) & (predictions != 2)]
errors

NameError: name 'X_test' is not defined